In [87]:
from manim import *
import numpy as np

In [88]:
# When the manim VGroup checks if an object is a mobject, it checks if it is
# either an instance of VMobject or OpenGLVMobject. I don't know the reasoning,
# but we will do the same here.
from manim.mobject.opengl.opengl_vectorized_mobject import OpenGLVMobject

class OrderedVGroup(VGroup):
    """ A VGroup that allows for ordered manipulation of its elements. 
    
    VGroups are already equipped with +, -, +=, -=, which are understood to
    add to the end of the group, and [] for elements retrieval/assignment. 
    
    This class adds:
     * insert(index, mobject) for inserting at a particular index. Returns `self`
        so that calls may be chained.
     * remove(index) and remove(mobject) for removing elements. Returns `self`
        so that calls may be chained.
     * pop(index) for removing and returning an element at a particular index.
     * __len__() and __iter__() over the elements of the group.


    Members of a VGroup must be unique, so adding in duplicate elements will 
    be ignored silently; use mobject.copy() if you want to add a duplicate. 
    
    remove(mobject) is a redundant method with - and -=, and due to uniqueness 
    remove(index) is too.

    Note: I'm not sure how much of the methods called are part of the public
    interface, so this may break in future versions of manim.
    """

    @staticmethod
    def _is_valid_mobject(item):
        """ Returns True if the given item is a valid mobject, as accepted by
            VGroup. """
        return isinstance(item, (VMobject, OpenGLVMobject))
    
    def _validate_mobjects(self, *mobjects):
        """ Checks the valididty of all provided mobjects and raises a value
            error if any are invalid. """
        if not all(self._is_valid_mobject(mobject) for mobject in mobjects):
            raise ValueError("Member mobjects must be of type VMobject")

    # Constructor is the same as VGroup
    def __init__(self, *submobjects, **kwargs):
        super().__init__(*submobjects, **kwargs)

    # a few aliases for methods called on members:
    def contains(self, mobject):
        """ Returns True if the given object is in the group. """
        # count method not needed since we are enforcing uniqueness
        return self.submobjects.count(mobject) > 0
    
    def index(self, mobject):
        """ Returns the index of the given mobject in the group. 
        
        Raises a ValueError if the mobject is not in the group: use 
        `contains(mobject)` to check if it is in the group first.
        """
        return self.submobjects.index(mobject)
    
    def __len__(self):
        """ Returns the number of elements in the group. """
        return len(self.submobjects)
    
    def __iter__(self):
        """ Returns an iterator over the elements of the group. """
        return iter(self.submobjects)
    
    # Insertion
    def insert(self, index, mobject):
        """ Inserts the given mobject at the given index. 
        
        Raises a ValueError if not given a valid mobject. Inserting a duplicate
        does nothing.
        """
        self._validate_mobjects(mobject)
        if not self.contains(mobject):
            self.submobjects.insert(index, mobject)

        return self
    
    # Removal
    def remove(self, mobject_or_index):
        """ Removes the given mobject or index from the group. 
        
        Raises a ValueError if not given a valid mobject or index.
        """
        if isinstance(mobject_or_index, int):
            self.pop(mobject_or_index)
        else:
            self._validate_mobjects(mobject_or_index)
            self -= mobject_or_index
        return self
    
    def pop(self, index):
        """ Removes and returns the mobject at the given index. """
        return self.submobjects.pop(index)

In [89]:
class LinearLayout(OrderedVGroup):

    @staticmethod
    def get_bounding_border_point(mobject, direction):
        """ Returns the point on the bounding box of the mobject in the given
        direction. Generalization of get_corner for any arbitrary direction, and
        return in the same coordinate system.
        """
        border_point = mobject.get_center()

        for ortho_dir in ((UP + RIGHT) / np.sqrt(2), (UP + LEFT) / np.sqrt(2)):
            component = np.dot(direction, ortho_dir)

            corner = mobject.get_corner(ortho_dir) - mobject.get_center()
            border_point += component * corner

        return border_point

    def __init__(self, *mobjects, direction = DOWN):
        VGroup.__init__(self, *mobjects)
        self.direction = direction
        
    def set_direction(self, direction):
        """ Sets the default direction for the layout. 

        This may be overridden by specific calls to arrange, and does not
        automatically update the layout- arrange must be called (manually or
        via an updater) to update the layout.
        """
        self.direction = direction
        return self

    def arrange(self, direction = None, root = None):
        """ Arranges the mobjects in the layout in the order they appear in 
        self.

        If direction is not provided, the instance's default direction is used.

        If root is provided, the first mobject will be placed at the given
        position. Otherwise, the first mobject will not be moved.
        """
        if direction is None:
            direction = self.direction

        if root is not None:
            self[0].move_to(root)

        for i in range(1, len(self)):
            align_edge_to = self.get_bounding_border_point(self[i-1], direction)
            border_vector = self.get_bounding_border_point(self[i], -direction) - self[i].get_center()
            self[i].move_to(align_edge_to - border_vector)

In [90]:
%%manim -qm -v WARNING TestBorderGeometry

class TestBorderGeometry(Scene):
    def construct(self):
        box = Polygon(LEFT, RIGHT, UP)
        angle = ValueTracker(0)
        self.add(box)

        point1 = always_redraw(
            lambda: Dot(
                box.get_boundary_point([np.cos(angle.get_value()), np.sin(angle.get_value()), 0]),
                color = RED)
        )
        self.add(point1)

        def get_boundary_point(direction):
            return LinearLayout.get_bounding_border_point(box, direction)
        point2 = always_redraw(
            lambda: Dot(
                get_boundary_point([np.cos(angle.get_value()), np.sin(angle.get_value()), 0]),
                color = BLUE)
        )
        self.add(point2)


        self.play(angle.animate.set_value(2*PI), run_time = 8)
        self.wait(3)

In [91]:
%%manim -ql -v WARNING TestLinearLayout

class TestLinearLayout(Scene):
    def construct(self):
        square = Square()
        circle = Circle()
        triangle = Triangle()

        layout = LinearLayout(square, triangle, circle)
        layout.scale(0.5)

        self.add(layout)
        self.wait(1)
        layout.arrange()
        self.wait(1)
        layout.arrange(direction = RIGHT)
        self.wait(1)
        layout.arrange(direction = UP + LEFT)
        self.wait(1)
        layout.arrange(root = 2 * UP + 2 * RIGHT)
        self.wait(1)
        layout.arrange(direction = RIGHT, root = ORIGIN)
        
        dir_angle = ValueTracker(0)
        dir_vec = lambda: np.array([np.cos(dir_angle.get_value()), np.sin(dir_angle.get_value()), 0])
        layout.add_updater(lambda l: l.arrange(direction = 2*dir_vec()))
        # draw direction vector
        align_vector = always_redraw(lambda: Arrow(ORIGIN,
            dir_vec(),
            buff = 0, color = RED))
        self.add(align_vector)

        # draw corner on circle
        corner = always_redraw(lambda: Dot(LinearLayout.get_bounding_border_point(layout[0], dir_vec())))
        self.add(corner)

        self.play(dir_angle.animate.set_value(PI), run_time = 5)

        layout.clear_updaters()
        layout.set_direction(RIGHT)
        layout.pop(1)
        self.wait(1)
        layout.arrange()
        self.wait(1)
        layout.insert(0, Square())
        layout.arrange()
        self.wait(1)

        l2 = layout.copy()
        l2.insert(1, Square())
        self.play(Transform(layout, l2), run_time = 4)
        self.wait(1)

        anims = []
        for m in layout:
            anims.append(Rotate(m, 2 * PI, about_point = m.get_center()))
        self.play(*anims, run_time = 5)
